In [872]:
DATA = """
Tile 2311:
..##.#..#.
##..#.....
#...##..#.
####.#...#
##.##.###.
##...#.###
.#.#.#..##
..#....#..
###...#.#.
..###..###

Tile 1951:
#.##...##.
#.####...#
.....#..##
#...######
.##.#....#
.###.#####
###.##.##.
.###....#.
..#.#..#.#
#...##.#..

Tile 1171:
####...##.
#..##.#..#
##.#..#.#.
.###.####.
..###.####
.##....##.
.#...####.
#.##.####.
####..#...
.....##...

Tile 1427:
###.##.#..
.#..#.##..
.#.##.#..#
#.#.#.##.#
....#...##
...##..##.
...#.#####
.#.####.#.
..#..###.#
..##.#..#.

Tile 1489:
##.#.#....
..##...#..
.##..##...
..#...#...
#####...#.
#..#.#.#.#
...#.#.#..
##.#...##.
..##.##.##
###.##.#..

Tile 2473:
#....####.
#..#.##...
#.##..#...
######.#.#
.#...#.#.#
.#########
.###.#..#.
########.#
##...##.#.
..###.#.#.

Tile 2971:
..#.#....#
#...###...
#.#.###...
##.##..#..
.#####..##
.#..####.#
#..#.#..#.
..####.###
..#.#.###.
...#.#.#.#

Tile 2729:
...#.#.#.#
####.#....
..#.#.....
....#..#.#
.##..##.#.
.#.####...
####.#.#..
##.####...
##..#.##..
#.##...##.

Tile 3079:
#.#.#####.
.#..######
..#.......
######....
####.#..#.
.#...#.##.
#.#####.##
..#.###...
..#.......
..#.###...
""".strip()

In [877]:
import numpy as np
import six

In [878]:
def transforms(x):      
    
    """
    Given a 2-dimensional np.array, it returns all the unique
    transformations of it which are a combination of rotations
    and flipping over the two axis.
    
    :param x: np.array
    :return: tuple(np.array,)
    """
    
    return (
        np.copy(x),
        np.flip(np.copy(x), 0),
        np.flip(np.copy(x), 1),
        np.flip(np.flip(np.copy(x), 0), 1),
        np.rot90(np.copy(x)),
        np.flip(np.rot90(np.copy(x)), 0),
        np.flip(np.rot90(np.copy(x)), 1),
        np.flip(np.flip(np.rot90(np.copy(x)), 0), 1),
    )

In [868]:
class Tile:
    
    """
    Represents a puzzle tile.
    """
    
    def __init__(self, number, layout):
        
        """
        Initiates the Tile object given its unique number
        and its internal 2-dimensional binary (0. and 1.) layout.
        
        :param number: int
        :param layout: np.array
        """
        
        self.number = number
        self.layout = layout
        self.width, self.height = len(self.layout[0]), len(self.layout)
        
    @property
    def withoutBorders(self):
        
        """
        Returns the layout of the tile without the external
        edges (i.e. the borders).
        
        :return: np.array
        """
        
        return self.layout[1:-1,1:-1]
        
    @property
    def state(self):
        
        """
        Returns a numeric identifier which uniquely identifies
        the tile based on its internal layout.
        
        :return: int
        """
        
        s = ''.join(['0' if not c else '1' for c in self.layout.flatten().tolist()])
        n = int(s, 2)
        return n
        
    def match(self, tile, i):
        
        """
        Determines whether the given tile can be a neighbour
        for the (self) tile at the given edge (0 for top, 1
        for right, 2 for bottom and 3 for left).
        
        :param tile: Tile
        :param i: int
        :return: bool
        """
        
        return np.all(self.edges[i] == tile.edges[(i+2)%4])
    
    @property
    def options(self):   
        
        """
        Yields all unique transformations (combinations of
        rotations and flipping) of the tile.
        
        :yield: Tile
        """
        
        for t in transforms(self.layout):
            yield Tile(self.number, t)
        
    @property
    def edges(self):
        
        """
        Returns the top, right, bottom and left edges of the tile.
        
        :return: tuple(np.array,)
        """
        
        return self.layout[0,:], self.layout[:,-1], self.layout[-1,:], self.layout[:,0]
        
    def __hash__(self):
        return hash((Tile, self.number))
    
    def __eq__(self, other):
        return hash(self) == hash(other)
    
    def __gt__(self, other):
        return self.number > other.number
    
    def __lt__(self, other):
        return self.number < other.number

In [889]:
class Solver:
    
    """
    Utility to solve AoC's 2020 20th task.
    """
    
    def __init__(self, data):
        
        """
        Initiates the utility given the puzzle input and finds
        the correct placement of all tiles.
        
        :param data: str
        """
        
        # First, the textual input data is parsed to store the
        # tile information;
        
        lines, i, tiles = data.splitlines(), 0, set()
        while i < len(lines):
            number = int(lines[i].split('Tile ')[1].split(':')[0])
            layout = np.array([[c == '#' for c in row] for row in lines[i+1:i+11]])
            tile = Tile(number, layout)
            tiles.add(tile)
            i += 12
        self.tiles = sorted(tiles)
        
        # The puzzle board properties are determined;
        self.n = int(np.sqrt(len(self.tiles)))
        self.width, self.height = self.tiles[0].width, self.tiles[0].height
        
        # Then, the board is constructed by finding a layout which
        # places all tiles in a valid format (i.e. all tiles match
        # their neighbours based on the shared edges);
        self._board = {}
        self._place(0, 0, set(self.tiles))
        
        # Finally, the full board is constructed by piecing together
        # the tiles (after having removed the outer edge of every tile);
        self.board = np.zeros( (((self.height-2) * self.n), ((self.width-2) * self.n)))
        for x, y in product(range(self.n), range(self.n)):
            self.board[y*(self.height-2):(y+1)*(self.height-2),x*(self.width-2):(x+1)*(self.width-2)] = self._board[(x,y)].withoutBorders

    def _place(self, x, y, tiles):
        
        """
        Recursively attempts to fill in the puzzle board by first placing
        a tile out of the given ones at the given location if it matches
        the neighbours (if any) and then repeating the process for the next
        tile in the layout.
        
        :param x: int
        :param y: int
        :param tiles: set(Tile,)
        """
        
        # First, all neighbours of the given locations are found;
        # this is because a tile can only be placed at a location
        # if its outer edges match the ones shared with the neighbours;
        
        neighbours = [
            self._board.get((x,y-1)),
            self._board.get((x+1,y)),
            self._board.get((x,y+1)),
            self._board.get((x-1,y)),
        ]
        
        # Iterate through every remaining tile in the filling process
        # and for every tile iterate over every transformation (a
        # combination of rotations and flipping);
        
        for i, tile in enumerate(tiles):
            for t in tile.options:               
                
                # Determine if the (transformed) tile is compatible
                # with the neighbours;
                
                for i, n in enumerate(neighbours):
                    if not n: continue
                    elif not t.match(n, i):
                        break
                else:
                    
                    # If the (transformed) tile is compatible, fill
                    # it in the board;
                    
                    self._board[(x, y)] = t
                    
                    # If it was the last tile to be found, return that
                    # it is completed;
                            
                    if x == self.n - 1 and y == self.n - 1 and len(tiles) == 1:
                        return True
                    
                    # Otherwise, find the next location to be inspected;
                    
                    if x == self.n - 1: nx, ny = 0, y + 1
                    else: nx, ny = x + 1, y
                        
                    # Attempt to place a tile at the next location (out
                    # of all the given tile without the newly placed one)
                    # and if it succeeds (down the line, as it's recursive),
                    # relay the result to the calling frame of the same function;
                    # however, if the puzzle could not be completed, do
                    # remove the newly placed tile and iteration will continue
                    # over the remaining the transformations of the tile
                    # and the remaining other tiles;
                        
                    if self._place(nx, ny, tiles.difference({t})): return True
                    else: self._board.pop((x, y))
                        
        # If none of the given tiles placed at the current location
        # led to the puzzle being completed (down the line), then
        # return the failure as this will allow backtracking to a 
        # previous addition and testing the next available option;
                    
        return False    
    
    def cornerProduct(self):
        return self._board[(0,0)].number * self._board[(self.n-1,0)].number * self._board[(0,self.n-1)].number * self._board[(self.n-1,self.n-1)].number
    
    def monster(self):
        
        """
        Finds the number of times the required pattern is found and
        how many potential locations on the board are never associated
        to a monster.
        
        :return: int, int
        """
        
        # Convert the pattern to a numpy.array as it will be
        # used to match against the board;
        
        pattern = """
                  # 
#    ##    ##    ###
 #  #  #  #  #  #   
        """
        c = pattern.count('#')
        pattern = '\n'.join(line for line in pattern.splitlines() if line.strip())
        pattern = [[1. if c == '#' else 0. for c in row] for row in pattern.splitlines()]
        pattern = np.array(pattern)
        ph, pw = pattern.shape
        bh, bw = self.board.shape
        
        # For every orientation of the board;
        
        for board in transforms(self.board):
            
            # The count of monsters found;
            count = 0
            
            # For every top-left coordinate of the window of the board
            # which can contain the pattern;
            
            for x, y in product(range(bw-pw+1),range(bh-ph+1)):
                # Extract the window;
                s = board[y:y+ph,x:x+pw]
                # Extract the values in the window for the location which
                # should contain the monster;
                z = s[pattern == 1.]
                # If all locations contain the monster character;
                if all(c == 1. for c in z.tolist()):
                    # Increase the count and replace those locations
                    # with a special value; this could technically
                    # lead to problems if a location is shared by
                    # multiple monsters, but it does not appear to be
                    # the case given the inputs;
                    count += 1
                    s[pattern == 1.] = 2.
                    board[y:y+ph,x:x+pw] = s
                
            if count:
                # If monsters were found, return how many and how
                # many potential monster locations are safe;
                z = sum(c == 1. for c in board.flatten().tolist())
                return count, z

In [890]:
s = Solver(DATA)

In [891]:
s.cornerProduct()

20899048083289

In [887]:
s.monster()

(2, 273)